In [85]:
import sys
import os
import matplotlib.image as img
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from skimage.io import imread, imshow
from skimage import filters
from tensorflow.keras import utils
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import *
from tensorflow.keras.utils import to_categorical
import tensorflow as tf
import numpy as np
import pandas
import multiprocessing
from scipy.cluster.vq import vq,kmeans,whiten
import glob
import random
import shutil
from keras.preprocessing import image
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from sklearn.decomposition import PCA



In [86]:
module_root_name = "AgeEstimator"
module_paths = [
    os.path.abspath(os.path.join('..')),
    os.path.abspath(os.path.join('../..')),
    os.path.abspath(os.path.join('../../..'))
]
module_paths = list(filter(lambda x: x.endswith(module_root_name), module_paths))
module_path = module_paths[0] if len(module_paths) == 1 else ""
if module_path not in sys.path:
    sys.path.append(module_path)

In [87]:
from server.data.dataset import DataLoader

In [106]:
vgg16_feature_list = []
def getPic(x_train):
    model = VGG16(include_top=True)
    for i in range(0, len(x_train)):
        pic = image.load_img(x_train[i], target_size=(224, 224))
        img_data = image.img_to_array(pic)
        
        img_data = np.expand_dims(img_data, axis=0)
        img_data = preprocess_input(img_data)

        vgg16_feature = model.predict(img_data)
        vgg16_feature_np = np.array(vgg16_feature)
        vgg16_feature_list.append(vgg16_feature_np.flatten())
    vgg16_feature_list_np = np.array(vgg16_feature_list)
    #data=whiten(train)
    print("load image done")
    
    pca = PCA(n_components=0.9)# 保证降维后的数据保持90%的信息
    vgg16_feature_list_np = pca.fit_transform(vgg16_feature_list_np)
    #vgg16_feature_list_np = pca.transform(vgg16_feature_list_np)
    print("PCA done")
    
    from sklearn.cluster import MiniBatchKMeans, KMeans
   
    from sklearn.metrics.pairwise import pairwise_distances_argmin 
    
    mbk = MiniBatchKMeans(n_clusters=16, init='k-means++', max_iter=100, batch_size=100, verbose=0, compute_labels=True, random_state=None, 
                    tol=0.0, max_no_improvement=10, init_size=None, n_init=3, reassignment_ratio=0.01)
    
    mbk.fit(vgg16_feature_list_np)
    mbk_means_cluster_centers = np.sort(mbk.cluster_centers_, axis = 0) 
    mbk_means_labels = pairwise_distances_argmin(vgg16_feature_list_np, mbk_means_cluster_centers) 
    
    # print the labels of each data 
    print("k-means done")
    fp = open('result.dat', 'w')
    i = 0
    for res in mbk_means_labels:
        fp.write(str(res))
        fp.write('\t')
        fp.write(str(y_train[i]))
        i = i + 1
        fp.write('\n')
    fp.close()
    model.save('mbk_model.h5')

In [107]:

if __name__ == '__main__':
    dl = DataLoader()
    x_train, y_train = dl.load_train()
    x_test, y_test = dl.load_test()
    
    
    getPic(x_train)

load image done
PCA done
k-means done
